# Data Warehousing

# TLDR

I took data from this relational database, put it in a S3 bucket, remodelled it in a star-schema and created this visualisation.

## What is a Data Warehouse?

To put in a very simple way, data warehouses are a central repository for the data in an organisation. Any business has many data sources (databases, excel and csv files, text data and so on), so a data warehouse is a repository where all this data is stored in a way that business users can easily understand it.

Of course that is not a simple task. Every team (marketing, accounting, sales, operations - the list goes on) has its own internal rules (business rules) on handling data. Therefore, building a Data Warehouse means applying all those rules in the background, so that data consumers get only the data in its most clean state. 

## The Task of a Data Warehouse

The main task of a data warehouse is, in this sense, to help data consumers make data-driven decisions. It's not an end in itself, but something that has to aggregate value to the business. 

## Data Modelling

In order to make data easier for end users to query, data warehouse often present data points in a Star-Schema or a Snowflake format. Both methods revolve around a fact table, which contains relevant business events, surrounded by dimension tables, which contain the entities that are important for data consumers. 

For example, the fact table sales would be linked to the dimension tables products and consumers, so that business users can check sales by customer or sales by product.

Data presented in this form is not normalized. This means that, differently from transactional databases, which avoid data repetition to ensure the integrity of its data, data warehouses allow for data repetition in order to make it more user friendly.

## Practical Example

To demonstrate how data warehousing works, I'll be working with [this dataset](https://www.kaggle.com/olistbr/brazilian-ecommerce) available on Kaggle. It reflects the data of an e-commerce business.

We can take a look at how the original database is structured, by looking at the following picture (located in the link above):

![mode](images/database_model.png)

*Note: a lot of the decisions towards creating and modelling a data warehouse are not to be made by data engineers, but by data consumers. They are the ones who have to explain the business logics. However, since I'm taking on this project on myself, I'll be making some assumptions on how the business is run.

We will suppose that the business analysts want to answer questions like the following:

1) What are the most popular product categories ?

2) Who are the best customers ? 

3) Who are the most popular sellers ? 

4) Where do I get most orders from ?

To understand the business process we can use the following diagram:

![logical](images/dw_logical.png)

This allows us to state the following:

**An order, all of which have an status, is placed by a customer out of a given location, who buys a product sold by a seller located at another location using a given payment method**

Now, we can take a look at the attributes of the entities we're dealing with, in order to find the grain of our data warehouse. That is the smallest relevant business event that we want to put in our fact table.

![relationanl](images/dw_relation.png)

Finally, we come to the conclusion that we want to track the sale of every product in an order. If we think about it that is quite logical.

An order may contain more than one product and the products contained in an order might come from different vendors. For this reason, in order to answer te business questions we made above, we have to track the sale of a product in an order (the order itself does not suffice)

*Note: we could define another grain (like payment methods or change of order status), if we were tackling other business questions. Ultimately, the modelling of a Data Warehouse depends on the questions we are trying to solve*

We can define the modell of our Data Warehouse as follows:

![physicall](images/physical_modell.png)

Now that the schema is defined, we can start our data warehouse.

# Staging the Data

In order to start working on our modell, I created a redshift cluster (which is an overkill for the data sample we have, but it's also free for two months in AWS), created a database called olist and a schema called staging. I also loaded some data there. If you want to know how I did, check out [this notebook](images/link to redshift).

# Creating the Dimensions

In order to make this notebook as short as possible, I'll pack the script to create the dimensional tables in another file in this folder. Please take a look at it, if there are any questions about how to create the dimension tables. For the date and time dimensions I  got some orientation from the [PostgreSQLWiki](https://wiki.postgresql.org/wiki/Date_and_Time_dimensions).

There is, however, one thing that I'd like to mention at this point: the distribution style of the tables in redshift. Redshift provides following [distribution styles](https://docs.aws.amazon.com/redshift/latest/dg/c_choosing_dist_sort.html): all, even and key (also auto, what basically means that redshift will try to sort what is the best distribution style for you).

- All: means that the table will be replicated across all clusters

- Even: means that the table will be evenly distributed across the clusters

- Key: the rows of a table are distributed according to the values in one column

Detailed guidelines on which one to choose are available in the [AWS Documentation for Redshift](https://docs.aws.amazon.com/redshift/latest/dg/t_designating_distribution_styles.html)

Basically, for most dimensions we will go with an even distribution (the distribution won't have a HUGE difference given the size of our data), but for the time and date dimensions I'll choose all. Again, this won't have a huge impact on performance for this example, but in a production environment this might make queries faster, because time and date columns are often used, but remain relatively small (so no big consequences for storage space) and don't change often.

# Fact Table

For the fact table, I'd like to apply some business logic. First, to determine wheter a product was sold or not, we will use the order_approved_at date (the order might be cancelled later, but that would be another fact, namely a cancelation). 

If we take a look at the order status for the sold items, we'll see the following configuration:

![order_status](images/dw_order_status.png)

Given this constellation, we will apply the following rules:

1) When the status is unavailable, we will put those records in a separate table so that business users can manually look at what went wrong

2) When the status is delivered, but there was no date of approval, the results will also be stored in an error table

3) Orders whose status is created won't be loaded (because they haven't been approved yet). The same goes for orders with the status cancelled.

The SQL for this would look like something like the facts:

![facts](images/dw_facts.png)

And for the error table

![errors](images/dw_errors.png)

The script for creating the fact table is also in the file linked above.

# Loading the Data

Now that we know how the structure of our database looks like and what data we have to put there, we can finally load data into the core tables.

Since we've put all the data points we need into the staging area, a simple SQL script will suffice for the first load, for example:

![redhisft_load_example](images/redshift_load_example.png)

We just repeat the procedure for the other tables, as shown in this [link zu script](link here):

*Note: if Redshift can't execute the commands related to the time and date dimensions, one possible workaround is to create the data in a regular postgres instance, upload the results to S3 and load the data from the S3 Bucket into Redshift*.

*This is due to a compatibility issue between the generate_series command from postgres, which isn't supported in redshift. Since the time and date dimensions are a "one-time-load-thing", it's ok to proceed this way*.

# Checking Results

Now, the most important part is to check if our architecture does solve the business problems we stated earlier:

## Popular Product Categories

![dw_prod](images/dw_prod.png)

## Best Customers

![dw_cust](images/dw_cust.png)

## Popular Sellers
![dw_seller](images/dw_sellers.png)

## States With Most Orders

![dw_states](images/dw_state.png)

## Conclusion

Everything looks good so far. We can find the answers we are looking for, so the architecture of the data warehouse is working great. If we look closely though, we can make the "product_category_english" column look a bit better. This data cleaning part could (should) have been implemented during the load of data, however I left it there on purpose in order to show how to clean it with PowerBi, as I show in [this file](link_to_file).